In [29]:
import os
import pandas as pd
import re

In [30]:
data_folder = 'data/'
dataframes = []

In [31]:
def extract_currency_code(col):
    if col in ['Date', 'Year']:
        return col
    match = re.search(r'\((.*?)\)', col)
    if match:
        return match.group(1)
    return col

In [32]:
# Iterate over all CSV files in the data folder
for filename in os.listdir(data_folder):
    if filename.endswith('.csv'):
        # Read the CSV file into a DataFrame
        file_path = os.path.join(data_folder, filename)
        df = pd.read_csv(file_path)
        
        # Extract the year from the filename (assuming the filename contains the year)
        year = filename.split('.')[0].split('_')[-1]  # Adjust this if the year is in a different part of the filename
        df['Year'] = int(year)
        
        # Append the DataFrame to the list
        dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a new CSV file (optional)
# combined_df.to_csv('data/combined.csv', index=False)


In [33]:
combined_df.fillna(0, inplace=True)

In [34]:
combined_df.columns = [extract_currency_code(col) for col in combined_df.columns]

In [35]:
combined_df['Date'] = pd.to_datetime(combined_df['Date'], format='%d-%b-%y')

In [36]:
import pandas as pd
from sqlalchemy import create_engine

# Assuming combined_df is already created and contains your data

# MySQL database connection details
username = 'root'
password = 'beansbestcat'
host = 'localhost'
port = '3307'  # Specify the port here
dbname = 'nt-t8-db'
table_name = 'historical_data'

# Create a connection string
connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{dbname}'

# Create a SQLAlchemy engine
engine = create_engine(connection_string)

# Write the DataFrame to the MySQL table
combined_df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)

print("DataFrame written to MySQL database successfully.")

DataFrame written to MySQL database successfully.
